In [ ]:
import os
import shutil
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bar

# Define the path to the main folder containing subfolders
main_folder = 'path/to/main/folder'

# Define the output folder where the separated images will be saved
output_folder = '/scratch/rharikr3/output/'  # Create separate folders for real and fake images here

# Create separate folders for real and fake images
real_folder = os.path.join(output_folder, 'real')
fake_folder = os.path.join(output_folder, 'fake')
os.makedirs(real_folder, exist_ok=True)
os.makedirs(fake_folder, exist_ok=True)

# Initialize progress bar
total_files = sum(len(files) for _, _, files in os.walk(main_folder))
progress_bar = tqdm(total=total_files, desc="Processing", unit="files")

# Iterate through each subfolder in the main folder
for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    if os.path.isdir(subfolder_path):
        # Find the CSV file within the subfolder
        csv_file = os.path.join(subfolder_path, 'metadata.csv')
        if os.path.exists(csv_file):
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)
            
            # Iterate through each row in the DataFrame
            for index, row in df.iterrows():
                image_path = os.path.join(subfolder_path, row['image_path'])
                target = row['target']
                
                # Check if the image exists
                if os.path.exists(image_path):
                    # Check the target label and move the image to the corresponding folder
                    if target == 0:
                        shutil.copy(image_path, real_folder)
                    elif target >= 1 and target <= 6:  # Assuming target labels for fake images range from 1 to 6
                        shutil.copy(image_path, fake_folder)
                    progress_bar.update(1)  # Update progress bar for each processed file
                else:
                    print(f"Image not found: {image_path}")
        else:
            print(f"CSV file not found in subfolder: {subfolder}")
    else:
        print(f"{subfolder} is not a directory.")

progress_bar.close()  # Close progress bar after processing

print("Images separated into real and fake folders.")
